In [1]:
#Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler,OneHotEncoder, LabelEncoder
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,calinski_harabasz_score, davies_bouldin_score
from sklearn.impute import SimpleImputer
import re
from mpl_toolkits.mplot3d import Axes3D


In [2]:
#Setting random state for reproducibility
rs = min(323001,328854) #since there are two students
np.random.seed(rs)
R = np.random.RandomState(rs)


In [3]:
km = KMeans(random_state=rs) 
df_tot=pd.read_csv('cla4lsp customers.csv',sep='\t')
df_tot

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,13-06-2013,46,709,...,5,0,0,0,0,0,0,3,11,0
2236,4001,1946,PhD,Together,64014.0,2,1,10-06-2014,56,406,...,7,0,0,0,1,0,0,3,11,0
2237,7270,1981,Graduation,Divorced,56981.0,0,0,25-01-2014,91,908,...,6,0,1,0,0,0,0,3,11,0
2238,8235,1956,Master,Together,69245.0,0,1,24-01-2014,8,428,...,3,0,0,0,0,0,0,3,11,0


In [7]:
#Sample 2/3 of the dataset 
workdf=df_tot.sample(frac=2/3,random_state=rs)
#check if percentage matches
workdf.shape[0]/df_tot.shape[0]
workdf.columns
workdf=workdf.drop(columns=["ID","Z_CostContact","Z_Revenue"])

In [8]:
#Randomly remove one column
def random_column():
    features_removable=[
    "MntWines", "MntFruits",
    "MntMeatProducts", "MntFishProducts", "MntSweetProducts", "MntGoldProds",
    "NumWebPurchases", "NumCatalogPurchases", "NumStorePurchases"
]
    R = np.random.RandomState(rs)
    random_index=R.randint(0,len(features_removable))
    return features_removable[random_index]
column_to_remove=random_column()
workdf=workdf.drop(columns=column_to_remove)

In [9]:
column_to_remove

'MntWines'

In [10]:
#check for missing values
workdf.isna().sum()

Year_Birth              0
Education               0
Marital_Status          0
Income                 17
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Response                0
dtype: int64

In [11]:
workdf = workdf.dropna(subset=['Income'])

In [12]:
# Now all null values have been removed. Let's check if some categorical attribute has been enconded with some string such as 'Unknown'
cat_col_to_check=[]
for column in workdf.columns:
    if pd.api.types.is_object_dtype(workdf[column]):
       cat_col_to_check.append(column)

In [13]:
#Let's check the unique values of these columns
for column in cat_col_to_check:
    print(column,workdf[column].unique())

Education ['Master' 'Graduation' 'Basic' 'PhD' '2n Cycle']
Marital_Status ['Married' 'Together' 'Single' 'Divorced' 'Alone' 'Widow' 'Absurd' 'YOLO']
Dt_Customer ['01-10-2012' '24-03-2013' '04-11-2013' '29-11-2013' '24-10-2013'
 '01-10-2013' '02-01-2013' '07-05-2013' '11-12-2012' '15-02-2014'
 '27-01-2014' '25-01-2014' '03-11-2012' '29-10-2012' '27-12-2013'
 '18-07-2013' '21-07-2013' '04-05-2014' '11-05-2013' '10-03-2013'
 '26-11-2013' '01-03-2014' '04-05-2013' '13-10-2012' '07-07-2013'
 '05-05-2013' '27-08-2012' '15-05-2013' '06-04-2013' '10-12-2013'
 '20-03-2013' '12-10-2013' '30-04-2014' '05-10-2013' '26-03-2013'
 '06-10-2013' '04-01-2014' '06-12-2012' '04-08-2012' '03-08-2012'
 '20-11-2012' '24-05-2014' '18-02-2013' '01-07-2013' '22-06-2013'
 '07-09-2012' '30-05-2013' '29-03-2013' '25-02-2014' '30-10-2013'
 '04-03-2013' '28-09-2012' '22-12-2012' '27-05-2013' '09-12-2013'
 '12-04-2013' '02-10-2013' '03-04-2014' '17-03-2014' '24-08-2012'
 '07-08-2013' '16-01-2013' '20-12-2013' '23-01-

In [14]:
#All the categorical attributes have a proper domain. We can proceed with the enconding of the categorical data.

# Exercise 2 (Encoding of Categorical Data)

In [15]:
#Ideas:

In [16]:
#Date: since date contain some ciclycal data (day and month repeat), we can split date in three variables: day,month,year.
#We can encode day and month with sine/cosine function, as 24:00 is close to 1:00 and December is close to January.
#We can preprocess education with label encoding, since there's a natural order between the education levels.
#We can encode marital_status with one-hot encoding, since the domain of values is limited and there's no order among values.
#We can proceed with the encoding phase:
enc_marital_status = OneHotEncoder(sparse=False)
enc_education = LabelEncoder()
enc_marital_status_df = pd.DataFrame(enc_marital_status.fit_transform(workdf[['Marital_Status']]),columns=enc_marital_status.categories_[0])
tmp_df=workdf.copy().reset_index(drop=True)
workdf=tmp_df.merge(enc_marital_status_df,left_index=True, right_index=True,how='left')
#TODO: datetime
#TODO: compute age

/home/omio/.local/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [17]:
workdf['Education_encoded']=enc_education.fit_transform(workdf['Education'])

In [18]:
workdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1476 entries, 0 to 1475
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year_Birth           1476 non-null   int64  
 1   Education            1476 non-null   object 
 2   Marital_Status       1476 non-null   object 
 3   Income               1476 non-null   float64
 4   Kidhome              1476 non-null   int64  
 5   Teenhome             1476 non-null   int64  
 6   Dt_Customer          1476 non-null   object 
 7   Recency              1476 non-null   int64  
 8   MntFruits            1476 non-null   int64  
 9   MntMeatProducts      1476 non-null   int64  
 10  MntFishProducts      1476 non-null   int64  
 11  MntSweetProducts     1476 non-null   int64  
 12  MntGoldProds         1476 non-null   int64  
 13  NumDealsPurchases    1476 non-null   int64  
 14  NumWebPurchases      1476 non-null   int64  
 15  NumCatalogPurchases  1476 non-null   i

In [19]:
enc_education.classes_

array(['2n Cycle', 'Basic', 'Graduation', 'Master', 'PhD'], dtype=object)

# Preprocessing

In [ ]:
#preprocessing of income , year of birth, Kidhome,Teenhome, MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth

In [ ]:
features=

In [ ]:
Xworkdf=workdf[features].copy()